In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [20]:
sys.path.append('../experiments/')
sys.path.append('../atlassampler/')

import default_args
import models
import plotting
import nuts
from atlassampler import Atlas
# from atlassampler.wrappers import cmdstanpy_wrapper


In [33]:
# Load default arguments

import argparse
parser = argparse.ArgumentParser()
parser = default_args.add_default_args(parser)
parser.add_argument('--combine_chains', type=int, default=0, help='combine nleap from chains')
args = parser.parse_args([])

## Override some arguments
args.n_chains = 4
args.exp = "rosenbrock"
args.n = 1
args.n_samples = 5000 
args.step_size = 0.1

print(args)



Namespace(exp='rosenbrock', n=1, suffix='', seed=999, n_leapfrog=20, n_samples=5000, n_burnin=100, target_accept=0.65, step_size=0.1, n_stepsize_adapt=100, offset=1.0, min_nleapfrog=3, max_nleapfrog=1024, n_leapfrog_adapt=100, low_nleap_percentile=10, high_nleap_percentile=90, nleap_distribution='uniform', constant_trajectory=2, probabilistic=1, n_hessian_samples=10, n_hessian_attempts=10, hessian_mode='bfgs', hessian_rank=-1, stepsize_distribution='lognormal', stepsize_sigma=1.2, max_stepsize_reduction=1000.0, delayed_proposals=1, metric='unit_e', n_metric_adapt=0, nuts=0, combine_chains=0, n_chains=4)


# Setup Stan Model

In [35]:
# Set some paths
SAVEFOLDER = '/mnt/ceph/users/cmodi/atlassampler/'
REFERENCE_FOLDER = "/mnt/ceph/users/cmodi/PosteriorDB/"
BRIDGESTAN = "/mnt/home/cmodi/Research/Projects/bridgestan/"
MODELDIR = '../'

reference_path =  f'{REFERENCE_FOLDER}/'
run_nuts = False #bool(not wrank)
model, D, lp, lp_g, ref_samples, files = models.stan_model(args.exp, args.n, 
                                                           bridgestan_path=BRIDGESTAN, 
                                                           model_directory=MODELDIR, 
                                                           reference_samples_path=reference_path,
                                                           run_nuts=run_nuts)

print("# of Dimensions", D)

..//stan/rosenbrock.data.json
{'D': 1}
Exception in loading reference samples:  [Errno 2] No such file or directory: '/mnt/ceph/users/cmodi/PosteriorDB///samples.npy'
# of Dimensions 2


# Run NUTS.

In [24]:
### Needs to setup cmdstanpy

# samples_nuts, sampler, step_size, n_leapfrogs_nuts = nuts.run_nuts(stanfile = files[0], 
#                                                                     datafile = files[1], 
#                                                                     args = args, 
#                                                                     # savefolder=savefolder_nuts, 
#                                                                     return_all=True)   

In [25]:
# plt.hist(samples_nuts[..., 0].flatten(), bins=50, density=True)
# plt.hist(ref_samples[..., 0].flatten(), bins=50, density=True, histtype='step', color='k', lw=2);

# ATLAS

In [36]:
kernel = Atlas(D, lp, lp_g, 
                      mass_matrix = np.eye(D), 
                      constant_trajectory = args.constant_trajectory,
                      probabilistic = args.probabilistic,
                      delayed_proposals = args.delayed_proposals,
                      offset = args.offset,
                      min_nleapfrog = args.min_nleapfrog,
                      max_nleapfrog = args.max_nleapfrog,
                      low_nleap_percentile = args.low_nleap_percentile,
                      high_nleap_percentile = args.high_nleap_percentile,
                      stepsize_distribution = args.stepsize_distribution,
                      nleap_distribution = args.nleap_distribution,
                      stepsize_sigma = args.stepsize_sigma,
                      n_hessian_samples = args.n_hessian_samples,
                      hessian_rank = args.hessian_rank,
                      max_stepsize_reduction = args.max_stepsize_reduction, 
                      hessian_mode = args.hessian_mode)
               


Distribution of trajectory lengths:  uniform


In [ ]:
from multiprocessing import Pool
args.n_samples = 10_000 

# Dummy function to call with pool for running multiple chains
def atlas_sampling(i):
    q0 = np.random.randn(D)
    q0 = model.param_unconstrain(q0)
    sampler = kernel.sample(q0,
                            seed = i,
                            n_leapfrog = args.n_leapfrog, 
                            step_size = args.step_size,
                            n_samples = args.n_samples, 
                            n_burnin = 50,
                            n_stepsize_adapt = 100,
                            n_leapfrog_adapt = args.n_leapfrog_adapt,
                            target_accept = args.target_accept,
                           )
    return sampler

n_chains = 4
with Pool(processes=n_chains) as pool: 
    results = pool.map(atlas_sampling, np.arange(n_chains))

Adapting step size for 100 iterationsAdapting step size for 100 iterationsAdapting step size for 100 iterationsAdapting step size for 100 iterations



Step size fixed to : Step size fixed to : Step size fixed to : Step size fixed to :     0.064150764904566990.070088835071835980.051666495345995950.04938312531439974



Adapting trajectory length for 100 iterationsAdapting trajectory length for 100 iterationsAdapting trajectory length for 100 iterationsAdapting trajectory length for 100 iterations



Shape of trajectories :  Shape of trajectories : (100,) 
(100,)average number of steps  : 
 Shape of trajectories : average number of steps  : 2.6797752808988764  
(100,)5.089887640449438

average number of steps  :  4.01875
Shape of trajectories :  (100,)
average number of steps  :  7.158536585365853


/mnt/home/cmodi/Research/Projects/AtlasSampler/atlassampler/algorithms/atlas.py:202: RuntimeWarning: divide by zero encountered in log
  log_prob_delayed = np.log((1-np.exp(log_prob_accept_ghost))) - np.log((1- np.exp(log_prob_accept_first)))
/mnt/home/cmodi/Research/Projects/AtlasSampler/atlassampler/algorithms/atlas.py:202: RuntimeWarning: divide by zero encountered in log
  log_prob_delayed = np.log((1-np.exp(log_prob_accept_ghost))) - np.log((1- np.exp(log_prob_accept_first)))


Iteration 0 of 10000Iteration 0 of 10000

Iteration 0 of 10000


/mnt/home/cmodi/Research/Projects/AtlasSampler/atlassampler/algorithms/atlas.py:202: RuntimeWarning: divide by zero encountered in log
  log_prob_delayed = np.log((1-np.exp(log_prob_accept_ghost))) - np.log((1- np.exp(log_prob_accept_first)))


Iteration 0 of 10000


In [ ]:
# Combine samples from different chains

s = np.stack([i.samples for i in results])
print(s.shape)

In [ ]:
# Plot histogram of first dim
plt.hist(s[..., 0].flatten(), bins=50, density=True, label='ATLAS samples')
plt.hist(ref_samples[..., 0].flatten(), bins=50, density=True, histtype='step', color='k', lw=2, label='Reference');
plt.legend()

In [ ]:
# Scatter plot

plt.plot(ref_samples[0, ::100, 0], ref_samples[0, ::100, 1], 'k.', alpha=0.5, label='Reference')
plt.plot(s[..., 0].flatten(), s[..., 1].flatten(), '.', alpha=0.7, label='ATLAS samples')
plt.legend()